In [1]:
# %% test cuda
import torch #type: ignore
torch.cuda.is_available()

True

In [9]:
a = lambda *x:None

In [10]:
a(5,4)

In [7]:
from tqdm import tqdm

class _LoaderDecorator:
    def __init__(self , raw_loader , progress_bar = True) -> None:
        self.loader = tqdm(raw_loader , total=len(raw_loader)) if progress_bar else raw_loader
        self.display_text = None

    def __len__(self): 
        return len(self.loader)

    def __iter__(self):
        for batch_i , batch_data in enumerate(self.loader):
            yield batch_data

    def add_text(self , display_text):
        self.display_text = display_text
        return self

    def display(self , **kwargs):
        if isinstance(self.display_text , str) and isinstance(self.loader , tqdm): 
            self.loader.set_description(self.display_text.format(**kwargs))

loader = list(range(10))
loader = _LoaderDecorator(loader).add_text('abc')

for a in loader:
    print(a)
    loader.display(c=5)



abc: 100%|██████████| 10/10 [00:00<00:00, 779.48it/s]

0
1
2
3
4
5
6
7
8
9


In [1]:
# %% test a specific model
from src import ModelTrainer
ModelTrainer.fit(process = 0 , resume = 0 , checkname= 1)

Use device name: NVIDIA GeForce RTX 4090


24-04-20 12:40:43|MOD:ModelModule |: Model Specifics:
24-04-20 12:40:43|MOD:ModelModule |: Start Process [Data] at Sat Apr 20 12:40:43 2024!


--Process Queue : Data + Fit + Test
--Start Training New!
--Model_name is set to gru_day_ShortTest.21!
{'random_seed': 10,
 'model_name': 'gru_day_ShortTest.21',
 'model_module': 'gru',
 'model_data_type': 'day',
 'model_data_prenorm': {'day': {'divlast': True, 'histnorm': True}},
 'labels': ['std_lag1_10'],
 'beg_date': 20170103,
 'end_date': 20171201,
 'interval': 120,
 'input_step_day': 5,
 'sample_method': 'train_shuffle',
 'shuffle_option': 'epoch'}
{'hidden_dim': [64],
 'seqlens': [{'day': 30, '30m': 30, 'dms': 30}],
 'tra_seqlens': [{'hist_loss': 40}],
 'dropout': [0.1],
 'enc_in': [True],
 'enc_att': [False],
 'rnn_type': ['lstm'],
 'rnn_att': [False],
 'rnn_layers': [2],
 'dec_mlp_layers': [2],
 'num_output': [1],
 'kernel_size': [3],
 'hidden_as_factor': [False],
 'ordered_param_group': [False]}
use /home/mengkjin/Workspace/learndl/data/torch_pack/day.20240313.pt


24-04-20 12:40:46|MOD:ModelModule |: Finish Process [Data], Cost 3.3 Secs
24-04-20 12:40:46|MOD:ModelModule |: Start Process [Fit] at Sat Apr 20 12:40:46 2024!


Pre-Norming method of [day] : {'divlast': True, 'histnorm': True}


Train Ep#  0 loss : 1.01523:   5%|▌         | 1/19 [00:00<00:04,  4.11it/s]

score function of [spearman] calculated and success!
loss function of [pearson] calculated and success!


Valid Ep#  0 score : 0.01508: 100%|██████████| 20/20 [00:00<00:00, 247.33it/s]
FirstBite Ep#  0 : loss  1.01249, train-0.01828, valid 0.01508, max 0.0151, best 0.0151, lr1.0e-07
Valid Ep#  1 score : -0.00542: 100%|██████████| 20/20 [00:00<00:00, 239.39it/s]
FirstBite Ep#  1 : loss  0.97275, train 0.04108, valid-0.00542, max 0.0151, best 0.0151, lr1.3e-03
Valid Ep#  2 score : 0.04450: 100%|██████████| 20/20 [00:00<00:00, 245.39it/s]
FirstBite Ep#  2 : loss  0.93790, train 0.06977, valid 0.04450, max 0.0445, best 0.0445, lr2.5e-03
Valid Ep#  3 score : 0.07741: 100%|██████████| 20/20 [00:00<00:00, 193.31it/s]
FirstBite Ep#  3 : loss  0.90524, train 0.09104, valid 0.07741, max 0.0774, best 0.0774, lr3.8e-03
Valid Ep#  4 score : 0.11452: 100%|██████████| 20/20 [00:00<00:00, 231.42it/s]
FirstBite Ep#  4 : loss  0.87768, train 0.11856, valid 0.11452, max 0.1145, best 0.1145, lr5.0e-03
Valid Ep#  5 score : 0.11745: 100%|██████████| 20/20 [00:00<00:00, 187.93it/s]
FirstBite Ep#  5 : loss  0.860

In [1]:
# prepare data
from src import DataModule
DataModule.prepare_data()

Use device name: NVIDIA GeForce RTX 4090


24-03-31 09:52:11|MOD:data_preprocessing|: if_train is False , Data Processing start!
24-03-31 09:52:11|MOD:data_preprocessing|: 3 datas :['y', 'trade_day', 'trade_30m']


Sun Mar 31 09:52:11 2024 : y start ...
labels blocks reading ret10_lag DataBase's ...... cost 0.39 secs
labels blocks reading ret20_lag DataBase's ...... cost 0.33 secs
labels blocks merging ...... cost 0.05 secs
models blocks reading risk_exp DataBase's ...... cost 1.66 secs
models blocks merging ...... cost 0.17 secs
y blocks process ...... cost 0.88 secs
y blocks masking ...... cost 0.05 secs
y blocks saving  ...... cost 0.14 secs
y blocks norming ...... cost 0.00 secs
Sun Mar 31 09:52:14 2024 : y finished! Cost 3.67 Seconds
Sun Mar 31 09:52:14 2024 : trade_day start ...
trade blocks reading day DataBase's ...... cost 0.56 secs
trade_day blocks merging ...... cost 0.00 secs
trade_day blocks process ...... cost 0.03 secs
trade_day blocks masking ...... cost 0.05 secs
trade_day blocks saving  ...... cost 0.16 secs
trade_day blocks norming ...... cost 0.00 secs
Sun Mar 31 09:52:15 2024 : trade_day finished! Cost 0.81 Seconds
Sun Mar 31 09:52:15 2024 : trade_30m start ...
trade blocks r

24-03-31 09:52:21|MOD:data_preprocessing|: Data Processing Finished! Cost 10.02 Seconds


... cost 1.13 secs
trade_30m blocks norming ...... cost 0.00 secs
Sun Mar 31 09:52:21 2024 : trade_30m finished! Cost 5.19 Seconds
